<a href="https://colab.research.google.com/github/mohamedelzaim77/first/blob/main/the%20code%20firstipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os
import shutil
import random

DATA_DIR    = r"/content/dataset"
NUM_CLASSES = 2
BATCH_SIZE  = 32
EPOCHS      = 10
LR          = 0.001
DEVICE      = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Data Reorganization Logic Start ---
# Define paths for reorganized data
source_base_dir = DATA_DIR
train_dir = os.path.join(DATA_DIR, "train")
val_dir = os.path.join(DATA_DIR, "val")

# Create train and val directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Get list of class folders (e.g., 'dogs_set', 'cats_set')
# Filter out 'train' and 'val' if they were created in a previous run but are empty
class_folders = [f for f in os.listdir(source_base_dir) if os.path.isdir(os.path.join(source_base_dir, f)) and f not in ['train', 'val']]

# Only reorganize if the class folders actually contain files and the train/val directories are not yet populated
if class_folders and not os.listdir(train_dir):
    for class_folder in class_folders:
        class_path = os.path.join(source_base_dir, class_folder)
        images = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
        random.shuffle(images)

        train_split_idx = int(0.8 * len(images))
        train_images = images[:train_split_idx]
        val_images = images[train_split_idx:]

        # Create class subdirectories in train and val
        os.makedirs(os.path.join(train_dir, class_folder), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_folder), exist_ok=True)

        # Move images to train directory
        for img_path in train_images:
            dest_path = os.path.join(train_dir, class_folder, os.path.basename(img_path))
            shutil.move(img_path, dest_path)

        # Move images to val directory
        for img_path in val_images:
            dest_path = os.path.join(val_dir, class_folder, os.path.basename(img_path))
            shutil.move(img_path, dest_path)

    print(f"Data reorganized into train and val directories within {DATA_DIR}")
else:
    print("Data already organized or no class folders found for reorganization.")
# --- Data Reorganization Logic End ---

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=DATA_DIR + "/train", transform=train_transform)
val_dataset   = datasets.ImageFolder(root=DATA_DIR + "/val",   transform=val_transform)

train_loader  = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader    = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False)

print(f"Classes : {train_dataset.classes}")
print(f"Train samples : {len(train_dataset)} | Val samples : {len(val_dataset)}")

model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

for param in model.parameters():
    param.requires_grad = False

in_features = model.fc.in_features
model.fc    = nn.Linear(in_features, NUM_CLASSES)
model       = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=LR)

for epoch in range(1, EPOCHS + 1):

    model.train()
    train_loss    = 0.0
    train_correct = 0

    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss    = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss    += loss.item() * images.size(0)
        train_correct += (outputs.argmax(dim=1) == labels).sum().item()

    train_loss /= len(train_dataset)
    train_acc   = train_correct / len(train_dataset)

    model.eval()
    val_loss    = 0.0
    val_correct = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss    = criterion(outputs, labels)
            val_loss    += loss.item() * images.size(0)
            val_correct += (outputs.argmax(dim=1) == labels).sum().item()

    val_loss /= len(val_dataset)
    val_acc   = val_correct / len(val_dataset)

    print(f"Epoch [{epoch}/{EPOCHS}]  "
          f"Train Loss: {train_loss:.4f}  Train Acc: {train_acc*100:.2f}%  |  "
          f"Val Loss: {val_loss:.4f}  Val Acc: {val_acc*100:.2f}%")

torch.save(model.state_dict(), "resnet50_finetuned.pth")
print("\nModel saved to resnet50_finetuned.pth")

Data reorganized into train and val directories within /content/dataset
Classes : ['cats_set', 'dogs_set']
Train samples : 800 | Val samples : 200
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:01<00:00, 74.5MB/s]


Epoch [1/10]  Train Loss: 0.3824  Train Acc: 89.38%  |  Val Loss: 0.1910  Val Acc: 99.00%
Epoch [2/10]  Train Loss: 0.1620  Train Acc: 97.62%  |  Val Loss: 0.1239  Val Acc: 99.00%
Epoch [3/10]  Train Loss: 0.1088  Train Acc: 97.62%  |  Val Loss: 0.0989  Val Acc: 99.50%
Epoch [4/10]  Train Loss: 0.0866  Train Acc: 98.50%  |  Val Loss: 0.0867  Val Acc: 99.50%
Epoch [5/10]  Train Loss: 0.0710  Train Acc: 98.88%  |  Val Loss: 0.0803  Val Acc: 99.50%
Epoch [6/10]  Train Loss: 0.0657  Train Acc: 98.25%  |  Val Loss: 0.0770  Val Acc: 99.50%
Epoch [7/10]  Train Loss: 0.0503  Train Acc: 99.62%  |  Val Loss: 0.0708  Val Acc: 99.50%
Epoch [8/10]  Train Loss: 0.0516  Train Acc: 98.88%  |  Val Loss: 0.0688  Val Acc: 99.50%
Epoch [9/10]  Train Loss: 0.0480  Train Acc: 99.00%  |  Val Loss: 0.0681  Val Acc: 99.50%
Epoch [10/10]  Train Loss: 0.0367  Train Acc: 99.75%  |  Val Loss: 0.0666  Val Acc: 99.50%

Model saved to resnet50_finetuned.pth


In [4]:
import zipfile
import os

zip_file_path = '/content/archive.zip'
output_dir = '/content/dataset'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f'Archive extracted to: {output_dir}')
print(f'Contents of {output_dir}: {os.listdir(output_dir)}')

Archive extracted to: /content/dataset
Contents of /content/dataset: ['dogs_set', 'cats_set']
